# Walkthrough del uso de `difusion_lib` 

Este cuaderno explica **cómo usar la librería `difusion_lib`** para:
- Difundir masa en grafos
- Analizar componentes conexas / fuertemente conexas
- Ejecutar el proceso de *pelado*
- **Generar un Dashboard 3D interactivo combinado**
- Visualizar y exportar resultados

> **Requisito**: la librería `difusion_lib` debe estar instalada o accesible en el `PYTHONPATH`.


## 1. Importaciones

Aquí **NO redefinimos clases**. Todo se importa directamente desde `difusion_lib`.


In [ ]:
import networkx as nx
import pandas as pd

# Importación correcta de la librería
from difusion_lib import (
    MotorDifusion,
    AnalizadorPelado,
    VisualizadorPelado,
    ControladorPelado
)


## 2. Crear un grafo de ejemplo

Creamos un grafo dirigido con varias componentes fuertemente conexas
y asignamos masa inicial a cada nodo.


In [ ]:
G = nx.DiGraph()

G.add_nodes_from(range(1, 7))
G.add_edges_from([
    (1, 2), (2, 3), (3, 1),
    (3, 4),
    (4, 5), (5, 4),
    (5, 6)
])

for n in G.nodes():
    G.nodes[n]['val'] = 1.0

G


## 3. Difusión de masa (`MotorDifusion`)

El motor redistribuye la masa de cada nodo entre sus vecinos.


In [ ]:
motor = MotorDifusion(G, tasa_difusion=0.7)
motor.ejecutar(iteraciones=50)

nx.get_node_attributes(G, 'val')


## 4. Análisis de componentes (`AnalizadorPelado`)

Calculamos métricas por componente (masa, tamaño, impacto global).


In [ ]:
metricas = AnalizadorPelado.obtener_metricas_cfc(
    G,
    version_pelado=0,
    total_nodos_original=len(G.nodes())
)

pd.DataFrame(metricas)


## 5. Visualización Estática y Dashboard Plotly (`VisualizadorPelado`)

Ahora el visualizador permite generar:
- **Imagen PNG** (Matplotlib)
- **Archivo GEXF** (Gephi)
- **Dashboard 3D Combinado** (Plotly con selector de capas)


In [ ]:
# Renderizado estático tradicional
VisualizadorPelado.renderizar(
    G,
    titulo="Ejemplo_Difusion_2D",
    ruta_base="salida_demo",
    exportar_gephi=True,
    mostrar_grafico=True
)

# NUEVO: Dashboard 3D Interactivo Combinado (Todas las capas en un solo gráfico)
VisualizadorPelado.renderizar_3d(
    G, 
    titulo="Dashboard_Difusion_3D", 
    ruta_base="salida_demo"
)


## 6. Proceso completo de pelado (`ControladorPelado`)

El controlador ahora genera automáticamente un único Dashboard interactivo que consolida todas las etapas del pelado.


In [ ]:
controlador = ControladorPelado(G)

registro = controlador.ejecutar_estudio_pelado(
    num_pelados=3,
    iteraciones_por_pelado=50,
    umbral_masa=2.5,
    tasa_difusion=0.7,
    valor_inicio=1.0,
    exportar_resultados=True,
    carpeta_exportacion="salida_estudio"
)

pd.DataFrame(registro)


## 7. Salidas generadas

La carpeta de salida ahora incluye el dashboard consolidado:
- `imagenes_grafos/` → Visualizaciones 2D (PNG)
- **`dashboard_3d_interactivo.html`** → Un solo archivo con todas las vistas seleccionables
- `archivos_gephi/` → Grafos para Gephi (GEXF)
- `reportes_datos/` → CSVs analíticos
- `reporte_resumen_pelado.csv` → Historial del pelado
